In [47]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import time
import pandas as pd
import numpy as np

In [54]:
product_df=pd.read_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트/제품정보.csv", encoding='CP949')
df=product_df['link']
df.count()
df

#driver= wd.Chrome(options=chrome_options)
#driver.get(df[69])

In [58]:
import math

#브라우저 꺼짐 방지 
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver= wd.Chrome(options=chrome_options)
review_tables=[]

for i in df:
    try:
        url="{}".format(i)
        driver.get(url)
        time.sleep(0.5)

        #리뷰
        driver.find_element(By.CLASS_NAME, 'goods_reputation').click()
        time.sleep(1)

        #최신순, 체험단 제외
        driver.find_element(By.CSS_SELECTOR, '#gdasSort > li:nth-child(3) > a').click()
        #driver.find_element(By.CSS_SELECTOR, '#searchType_3').click()


        #페이지 넘어가며 리뷰 긁어오기

        #리뷰 총 개수
        count=driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.total > em').text
        count=count.replace(",","")
        count=int(count)
        stop=math.ceil(count/10)

        for num in range(1,stop+1): #페이지 넘어가기  
            if 1 < num < 11: 
                driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(num)).click() 
            elif num ==11:
                driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next').click()
            elif num > 20: ##리뷰 200개까지만 가져오기
                break     
            elif num >=12:
                if (num%10)==0:
                    driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child(11)').click()
                elif ((num-11)%10)==0: #다음버튼
                    driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next').click() 
                else: driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(num%10+1)).click()   

            try: #페이지 당 리뷰 가져오기
                for i in range(1,11): 
                    time.sleep(1) #대기안하면 오류 생김 
                    data_table={} #리뷰에서 필요한 내용들 가져오기
                    data_table['제품명']=driver.find_element(By.CLASS_NAME, 'prd_name').text
                    data_table['ID']=driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child({}) > div.info > div > p > a'.format(i)).text 
                    try: 
                        data_table['피부타입']=driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child({}) > div.info > div > p.tag'.format(i)).text
                    except:
                        data_table['피부타입']=np.nan
                    data_table['별점']=driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child({}) > div.review_cont > div.score_area > span.review_point > span'.format(i)).text
                    data_table['날짜']=driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child({}) > div.review_cont > div.score_area > span.date'.format(i)).text 
                
                    #상세 평점(수치형으로 바꾸는게? 타입 겹치는지 확인?)
                    try:
                        data_table['상세 평점(1)']=driver.find_element(By.XPATH, '//*[@id="gdasList"]/li[{}]/div[2]/div[2]/dl[1]/dd/span'.format(i)).text
                        data_table['상세 평점(2)']=driver.find_element(By.XPATH, '//*[@id="gdasList"]/li[{}]/div[2]/div[2]/dl[2]/dd/span'.format(i)).text
                        data_table['상세 평점(3)']=driver.find_element(By.XPATH, '//*[@id="gdasList"]/li[{}]/div[2]/div[2]/dl[3]/dd/span'.format(i)).text
                    except:
                        data_table['상세 평점(1)']=np.nan
                        data_table['상세 평점(2)']=np.nan
                        data_table['상세 평점(3)']=np.nan

                    data_table['리뷰 내용']=driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child({}) > div.review_cont > div.txt_inner'.format(i)).text

                    review_tables.append(data_table)
            except:
                print("끝") #마지막 페이지 리뷰가 10개가 아닌 경우 나옴
                break
    except Exception as e:
        print("에러발생",e)
        continue

review_df=pd.DataFrame.from_dict(review_tables)
review_df

,제품명,ID,피부타입,별점,날짜,상세 평점(1),상세 평점(2),상세 평점(3),리뷰 내용
0,비욘드 엔젤아쿠아 소프트 페이셜 필링젤 100ml 1+1 디즈니 에디션,sh26227,NaN,5점만점에 5점,2023.12.14,복합성에 좋아요,아주 만족해요,자극없이 순해요,쓰던 제품 다써서 새로 구입해봤어요~~~\n때처럼 잘 밀려 나오는 제품이 좋은데\n...
1,비욘드 엔젤아쿠아 소프트 페이셜 필링젤 100ml 1+1 디즈니 에디션,17k****,NaN,5점만점에 5점,2023.12.13,건성에 좋아요,아주 만족해요,자극없이 순해요,세안시 자극없이 스크럽 되어서 좋고 향도 순하고 만족입니다
2,비욘드 엔젤아쿠아 소프트 페이셜 필링젤 100ml 1+1 디즈니 에디션,닝닝닝이,복합성 겨울쿨톤 모공 탄력,5점만점에 5점,2023.12.12,복합성에 좋아요,보통이에요,자극없이 순해요,원플러스원이라 2개나 쟁여둘 수 있네요. 내년까지 잘 써야겠어요.
3,비욘드 엔젤아쿠아 소프트 페이셜 필링젤 100ml 1+1 디즈니 에디션,냠냠냠냠냠미,건성 웜톤 각질 민감성,5점만점에 5점,2023.12.10,복합성에 좋아요,보통이에요,보통이에요,여태 써본 필링젤 중에 제일 순한 것 같아요.\n다른 유명 제품들보다 이게 더 좋은...
4,비욘드 엔젤아쿠아 소프트 페이셜 필링젤 100ml 1+1 디즈니 에디션,smi****,NaN,5점만점에 5점,2023.12.09,복합성에 좋아요,보통이에요,자극없이 순해요,세일해서 저렴하게 잘삿어용\n후뚜루 마뚜루 쓰기 좋아요\n주 2회정도 하면 딱 좋아...
...,...,...,...,...,...,...,...,...,...
465,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,nandd****,NaN,5점만점에 5점,2023.02.18,NaN,NaN,NaN,"토너패드계 은둔고수..? 사실 리뷰는 잘 안쓰는데, 이건 사용해보니 정말 좋아서 남..."
466,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,sot****,NaN,5점만점에 5점,2023.02.18,NaN,NaN,NaN,토너가 똑같다생각하고 어지러운 화장대 에 올라가도 깔끔한 디자인 찾아봐야지 해서 우...
467,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,s9****,NaN,5점만점에 5점,2023.02.18,NaN,NaN,NaN,한통 구입해 쓰고 있는데 너무 좋아 동생주려고 남편찬스로 선물로 받음 ㅋ 겨울내내 ...
468,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,krl****,NaN,5점만점에 5점,2023.02.16,NaN,NaN,NaN,토너패드 처음 써봤어요^^\n저는 피부가 극예민성에 악건성 이거든요 그래서 아무 화...


In [59]:
review_df['제품명'].value_counts()

제품명
비욘드 엔젤아쿠아 소프트 페이셜 필링젤 100ml 1+1 디즈니 에디션                      200
[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+60매 리필증정 기획)    190
플리프 시카-알로에 카밍 올인원 로션 200ml (트러블 피부용)                          80
Name: count, dtype: int64

In [35]:
review_df['제품명'].unique()

array(['코스노리 비건 아보카도 아이크림 (본품30ml+15ml 증정)', '쏘내추럴 비건 메이크업 캄 픽서 100ml',
       '클라뷰 리얼 비건 저분자 콜라겐 탄력 앰플 30ml', '[대용량] 나인위시스 비건 하이드라 앰플 토너 500ml',
       '클라뷰 리얼 비건 저분자 콜라겐 탄력 크림 50ml', '허블룸 콤부차 비건 콜라겐 크림 50ml',
       '[화해 1위] 나인위시스 비건 화이트 톤업 크림 50ml 더블기획',
       '[화해 1위] 나인위시스 비건 화이트 톤업 크림 50ml',
       '[1+1/단독기획] 나인위시스 비건 하이드라 앰플 수딩 크림 50ml 더블기획',
       '비얼라우 무화과,부활초 비건 올인원 200g+30g 기획',
       '[선물추천] 달바 시그니처 오리지널 & 프리미엄 비건 미스트 세럼 기프트 세트',
       '비욘드 엔젤아쿠아 보습장벽크림 150ml 1+1기획 (비건)',
       '비욘드 엔젤아쿠아 수분 진정 토너 500ml (비건)',
       '비욘드 엔젤아쿠아 워터 에센스 인 로션 200ml (비건 에센스인로션)',
       '비욘드 엔젤아쿠아 수분 진정 크림 150ml 1+1 기획 (비건)',
       '비얼라우 콤부차,레몬 비건 올인원 200g+30g 기획', '비욘드 엔젤아쿠아 스킨케어 2종세트(비건)',
       '빌리프 프라블럼 솔루션 비건 스팟 젤 15ml', '더바디샵 에델바이스 인텐스 크림 50ml(비건)',
       '더랩바이블랑두 엑스퍼트루 모공탄력 세럼 30ml 기획 (+샤셰 4매 증정)',
       '[이동욱PICK]더마펌 수분보습 진정 크림 50ml+50ml 기획',
       '[이동욱PICK] 더마펌 수딩 리페어 토너 R4 200ml+200ml 기획',
       '[이동욱PICK]더마펌 흔적끝 토닝 세럼 30ml+30ml 기획 (+마스크팩 1매증정)',
       '플리프 시카-알로에 카밍 올인원

In [31]:
review_df['제품명'].nunique()

65

In [61]:
review_data1 = pd.read_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트/리뷰1.csv", encoding='utf-8-sig')
review_data2 = pd.read_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트/리뷰2.csv", encoding='utf-8-sig')
review_data3 = pd.read_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트/리뷰3.csv", encoding='utf-8-sig')

In [62]:
result=pd.concat([review_data1, review_data2, review_data3])
result=result.reset_index()
result=result.drop('index', axis=1)
result

,Unnamed: 0,제품명,ID,피부타입,별점,날짜,상세 평점(1),상세 평점(2),상세 평점(3),리뷰 내용
0,0,[비건인증] 아리얼 스트레스 릴리빙 미셀라 립앤아이 리무버 300ml,yoni****,NaN,5점만점에 5점,2023.12.17,건성에 좋아요,아주 만족해요,자극없이 순해요,아이리무버는 이것만 써요\n눈에 자극 없이 순해서 좋아요!!
1,1,[비건인증] 아리얼 스트레스 릴리빙 미셀라 립앤아이 리무버 300ml,yoni****,NaN,5점만점에 5점,2023.12.17,건성에 좋아요,아주 만족해요,자극없이 순해요,피부 좀 예민한 편인데 트러블 없이 잘 지워져요\n세일할 때 무조건 쟁여 두는 것 ...
2,2,[비건인증] 아리얼 스트레스 릴리빙 미셀라 립앤아이 리무버 300ml,jyjw****,NaN,5점만점에 5점,2023.12.16,건성에 좋아요,아주 만족해요,보통이에요,일케 깨끗-하게 지워지는 리무버 잘 없습니당 !! 재구매 의사 이따시
3,3,[비건인증] 아리얼 스트레스 릴리빙 미셀라 립앤아이 리무버 300ml,노래하는나잉,건성,5점만점에 5점,2023.12.16,복합성에 좋아요,아주 만족해요,자극없이 순해요,이 제품은 지난번에도 사용했었는데 휴대용으로 먼저 사용해 보았을때 눈가에 너무 자극...
4,4,[비건인증] 아리얼 스트레스 릴리빙 미셀라 립앤아이 리무버 300ml,dk****,NaN,5점만점에 5점,2023.12.15,건성에 좋아요,아주 만족해요,자극없이 순해요,향은 딱히 안 나고 눈 닦을때 패드가 말라있으면 눈 닦을때 아파서 물을 묻히고 닦는...
...,...,...,...,...,...,...,...,...,...,...
16738,465,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,nandd****,NaN,5점만점에 5점,2023.02.18,NaN,NaN,NaN,"토너패드계 은둔고수..? 사실 리뷰는 잘 안쓰는데, 이건 사용해보니 정말 좋아서 남..."
16739,466,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,sot****,NaN,5점만점에 5점,2023.02.18,NaN,NaN,NaN,토너가 똑같다생각하고 어지러운 화장대 에 올라가도 깔끔한 디자인 찾아봐야지 해서 우...
16740,467,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,s9****,NaN,5점만점에 5점,2023.02.18,NaN,NaN,NaN,한통 구입해 쓰고 있는데 너무 좋아 동생주려고 남편찬스로 선물로 받음 ㅋ 겨울내내 ...
16741,468,[민가든PICK/단독기획] 오어스 비건 민들레 히알루론 각질 토너패드 60매 (+6...,krl****,NaN,5점만점에 5점,2023.02.16,NaN,NaN,NaN,토너패드 처음 써봤어요^^\n저는 피부가 극예민성에 악건성 이거든요 그래서 아무 화...


In [69]:
result.to_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트/리뷰정보.csv", encoding='utf-8-sig')